# Creating a Deep Model to predict the antidepressant effect

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt # For data viz
import pandas as pd
import numpy as np
import sys

print('System Version:', sys.version)
print('PyTorch version', torch.__version__)
print('Numpy version', np.__version__)
print('Pandas version', pd.__version__)

System Version: 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]
PyTorch version 2.5.1
Numpy version 1.26.4
Pandas version 2.2.3


In [9]:
# Confirm device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")
try: 
    name = torch.cuda.get_device_name(0)
    count = torch.cuda.device_count()
    print(f"Device count: {count}")
    print(f"Device name: {name}")
except RuntimeError:
    print('No GPUs detected')

Using cuda device
Device count: 1
Device name: NVIDIA L40S


In [10]:
# Load the data
X_train = np.load('data/X_SYN.npy')
X_test = np.load('data/X_TEST_RAW.npy')

y_train = np.load('data/y_SYN.npy')
y_test = np.load('data/y_TEST_RAW.npy')


In [11]:
# Define the dataset
class COPEDataset(Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        cope_data = self.data[index]
        cope_data = (cope_data - np.min(cope_data)) / (np.max(cope_data) - np.min(cope_data))  # normalize

        label = self.target[index]
        volume = torch.tensor(cope_data, dtype=torch.float32).unsqueeze(0)  # (1, 91, 109, 91)
        label = torch.tensor([1.0, 0.0] if label == 0 else [0.0, 1.0], dtype=torch.float32)
        return volume, label

In [12]:
# Initiate the dataset and data loader
train_dataset = COPEDataset(X_train, y_train)
test_dataset = COPEDataset(X_test, y_test)
train_dataloader = DataLoader(train_dataset, batch_size=10)
test_dataloader = DataLoader(test_dataset, batch_size=10)

In [ ]:
# Define the model
class BrainClassifier3D(nn.Module):
    def __init__(self):
        super(BrainClassifier3D, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv3d(1, 16, kernel_size=3, stride=2, padding=1),   # volume (16, 46, 55, 46)
            nn.MaxPool3d(3, stride=2),                              # volume (16, 22, 27, 22)
            #nn.BatchNorm3d(16),
            nn.ReLU(),

            nn.Conv3d(16, 32, kernel_size=3, stride=2, padding=1),  # volume (32, 11, 14, 11)
            nn.MaxPool3d(3, stride=2),                              # volume (32, 5, 6, 5)
            #nn.BatchNorm3d(32),
            nn.ReLU(),

            #nn.Conv3d(32, 64, kernel_size=3, stride=2, padding=1),  #  (64, 6, 7, 6)
            #nn.MaxPool3d(3, stride=2),
            #nn.BatchNorm3d(64),
            #nn.ReLU()

            #nn.Conv3d(64, 128, kernel_size=3, stride=2, padding=1),  #  (128, 6, 7, 6)
            #nn.MaxPool3d(3, stride=2),
            #nn.BatchNorm3d(128),
            #nn.ReLU()
        )

        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Linear(32 * 5 * 6 * 5, 256),
            nn.ReLU(),
            #nn.Dropout(0.3),
            nn.Linear(256, 2),  # 2 output classes
            nn.Softmax()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x  # logits


In [24]:
# Setup
model = BrainClassifier3D().cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

# Train
for epoch in range(20):
    model.train()
    total_loss, correct = 0.0, 0

    for batch in train_dataloader:
        inputs, labels = batch
        inputs, labels = inputs.cuda(), labels.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == torch.argmax(labels, dim=1)).sum().item()

    acc = correct / len(train_dataloader.dataset)
    print(f"Epoch {epoch+1}, Loss: {total_loss:.2f}, Accuracy: {acc:.4f}")


Epoch 1, Loss: 32.48, Accuracy: 0.5057
Epoch 2, Loss: 31.14, Accuracy: 0.5057
Epoch 3, Loss: 31.07, Accuracy: 0.5057
Epoch 4, Loss: 30.91, Accuracy: 0.5057
Epoch 5, Loss: 30.90, Accuracy: 0.5057
Epoch 6, Loss: 30.92, Accuracy: 0.5034
Epoch 7, Loss: 30.78, Accuracy: 0.4989
Epoch 8, Loss: 30.90, Accuracy: 0.3977
Epoch 9, Loss: 30.80, Accuracy: 0.4046
Epoch 10, Loss: 30.82, Accuracy: 0.4207
Epoch 11, Loss: 30.76, Accuracy: 0.4069
Epoch 12, Loss: 30.77, Accuracy: 0.4023
Epoch 13, Loss: 30.73, Accuracy: 0.3931
Epoch 14, Loss: 30.73, Accuracy: 0.3954
Epoch 15, Loss: 30.68, Accuracy: 0.4000
Epoch 16, Loss: 30.72, Accuracy: 0.3793
Epoch 17, Loss: 30.69, Accuracy: 0.3701
Epoch 18, Loss: 30.65, Accuracy: 0.4023
Epoch 19, Loss: 30.64, Accuracy: 0.4184
Epoch 20, Loss: 30.65, Accuracy: 0.4092


In [ ]:
# Test the model

def evaluate(model, dataloader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            print(f'Predicted: {preds.item()}   True: {torch.argmax(labels, dim=1).item()}')
            correct += (preds == torch.argmax(labels, dim=1)).sum().item()
    return correct / len(dataloader.dataset)

val_acc = evaluate(model, test_dataloader)
print(f"Validation Accuracy: {val_acc:.4f}")
